In [1]:

from battle import *
from numpy.random import random, randint
from termcolor import colored
# (position=[0,0])
chessCanBeUsed = [ant,
                  llama,
                  wolf, 
                  ladybug, 
                  bee, 
                  swallower, 
                  sea_hedgehog, 
                  heal_deer, 
                  hippo, 
                  bear, 
                  butterfly, 
                  fireworm, 
                  mantis, 
                  scorpion, 
                  anglerfish, 
                  tiger, 
                  shark, 
                  spider, 
                  rabbit, 
                #   anglerfish(position=[0,0])
                  
                  ]
cost1Card = [ant, littleUglyFish, rabbit]
cost2Card = [llama, wolf, ladybug, bee, swallow, sea_hedgehog]
unavail = {electric_eel,monoceros,turtle,elephant,unicorn_b,littleUglyFish,monkey} # octopus
# 初始棋子
redTeam = []
blueTeam = []
class Player:
  # id = 0
  def __init__(self, id) -> None:
    # self.id = Player.id
    # Player.id += 1
    self.id = id
    self.round:int = 1
    self.chesses: dict[int, chessInterface] = {}
    self.chessInHand: list[chessInterface] = [] # max Of 5 cards
    self.chessOnField:list[chessInterface] = []
    self.goldAvail = 1
    self.buffs = []
    self.population = 1
    self.roundToNextPop = 1
    self.goldToNextPop = 1
    self.cost2Chance = 0 # 抽到2星卡的概率
    self.hp = 100

  def __repr__(self) -> str:
    if self.id == 0:
      color = 'red'
    else: 
      color = 'blue'
    return f"{colored('玩家'+str(self.id),color)}"
  
  def printStatus(self) -> None:
    print (f"\n" + \
          f"当前回合 {self.round}, 剩余血量：{self.hp}\n" + \
          f"{self} 现在拥有 {colored(self.population,'green')} 人口，距离下次升级还需要{colored(self.roundToNextPop,'light_cyan')}回合，或者{colored(self.roundToNextPop*self.roundToNextPop,'light_cyan')}金钱；\n" + \
          f"现在拥有的棋子为{self.chesses}\n" + \
          f"备战区棋子：{self.chessInHand}，上场棋子{self.chessOnField}；\n"+ \
          f"当前回合还有可用金钱{colored(self.goldAvail, 'yellow')}；\n" + \
          f"玩家buff有{self.buffs}；\n" + \
          "\n")

  def next_round(self):
    '''每回合开始阶段调用这个方法'''
    self.hp -= 10
    self.round += 1
    self.goldAvail = deepcopy(self.round)
    self.checkPopulationUpdate()      
    self.printStatus()  
  
# 金钱部分
  def reduce_gold(self, howMuch, purpose: str):
    '''花钱的时候调用这个方法'''
    if self.goldAvail - howMuch >= 0:
      self.goldAvail -= howMuch
      print(self,purpose,f"{colored('成功','green')},共花费: ",howMuch," 金币，还剩 ",self.goldAvail," 金币。")
      return True
    else:
      print(self,purpose,f"{colored('失败','red')},需要: ",howMuch,' 金币，现有 ',self.goldAvail,f" 金币，差{howMuch - self.goldAvail}金币。")
      return False
    
  def checkPopulationUpdate(self):
    """检查人口是否上升"""
    self.roundToNextPop -= 1
    self.goldToNextPop -= self.population
    if self.roundToNextPop == 0:
      self.population += 1
      self.roundToNextPop = deepcopy(self.population)
      self.costForNextPop = deepcopy(self.population)
      print(f"玩家{self.id} 现在拥有 {colored(self.population,'green')} 人口，升级至下一人口还需要{colored(self.roundToNextPop,'light_cyan')}回合，或者 {colored(self.roundToNextPop*self.roundToNextPop,'light_cyan')} 金钱；\n")
      
# 人口部分
  def upgrade_pop(self):
    """用金币升级人口"""
    if self.reduce_gold(purpose=f"升级人口", howMuch=self.population):
      self.checkPopulationUpdate()
      
# 棋子部分
  def printChesses(self):
    print(self.chesses)
    
  def get_random_chess_to_draw(self):
    """每回合随机抽卡"""
    cardList = []
    for i in range(3):
      if random() < self.cost2Chance:
        cardList.append( cost2Card[randint(0,len(cost2Card))] )
      else:
        cardList.append( cost1Card[randint(0,len(cost1Card))] )
    print(self,"本次选择有",cardList)
    return cardList
  def redraw(self):
    """重新抽卡"""
    if self.reduce_gold(purpose=f'刷新棋子', howMuch = 3):
      self.get_random_chess_to_draw()
    
  def checkChessID(self, chessID:int) -> chessInterface:
    print(chessID, type(chessID),self.chesses)
    try:
      chess = self.chesses[chessID]
    except:
      raise KeyError(f"{self}没有这张卡")
    return chess

  def get_chess(self,newChess: chessInterface):
    """购买棋子"""
    newChess.team = self.id
    if self.reduce_gold(purpose=f"购买棋子{newChess}", howMuch=newChess.star):
      self.chessInHand.append(newChess)
      self.chesses[newChess.uniqueID] = newChess 

  def sell_chess(self, chessID: int):
    """出售棋子"""
    chess = self.checkChessID(chessID=chessID)
    if self.chesses[chess]:
      self.chessOnField.remove(chess)
      self.goldAvail += chess.star/2
      print(f"{self}:已被卖掉，从chessOnField and chesses中移除,{self}获得金钱{chess.star/2}，现在闲置金钱{self.goldAvail}")
    else:
      self.chessInHand.remove(chess)
      self.goldAvail += chess.star/2
      print(f"{self}:已被卖掉，从chessInHand and chesses中移除,{self}获得金钱{chess.star/2}，现在闲置金钱{self.goldAvail}")
    for (s, c) in self.chesses.items():
      if c == chess:
        break
    del self.chesses[s]
    print(  f"       现在拥有的棋子为{self.chesses}")
    print(  f"       备战区棋子：{self.chessInHand}，\n  上场棋子{self.chessOnField}；")
      
  def hand_to_field(self, chessID: int, position: list[int]):
    """上场棋子"""
    
    chess = self.checkChessID(chessID=chessID)
    if chess in self.chessOnField:
      print(f"棋子{chess}已经在场上了，请勿重复上场!")
      return
    self.chessInHand.remove(chess)
    self.chessOnField.append(chess)
    chess.setInitialPosition(position = position)
    chess.onField = True
    print(f"{self}:{chess}上场到位置{colored(str(chess.position), color='magenta')}")
    print(f"       备战区棋子：{self.chessInHand}，\n       上场棋子{self.chessOnField}；")
    
  def field_to_hand(self, chessID: int):
    """下场棋子"""
    chess = self.checkChessID(chessID=chessID)
    if chess in self.chessInHand:
      print(f"棋子{chess}已经在备战区了，不能下场！")
      return
    chess.position = [-1,-1]
    self.chessOnField.remove(chess)
    self.chessInHand.append(chess)
    chess.onField = False
    print(f"{self}:{chess}下场")
    print(f"       备战区棋子：{self.chessInHand}，\n       上场棋子{self.chessOnField}；")
    
  
# 棋子升级
  def first_2_copy(self, chessID: int) -> list[chessInterface]:
    chess = self.checkChessID(chessID=chessID)
    sameChess = []
    for c in self.chesses.values():
      if c.id == chess.id:
        sameChess.append(c)
    if len(sameChess) >= 2:
      return sameChess[:2]
    else:
      return []

  def upgrade_chess(self, chessID: int ):
    chess = self.checkChessID(chessID=chessID)
    sameChessList = self.first_2_copy(chessID=chessID)
    if len(sameChessList) >= 2:
      for c in sameChessList:
        if c.onField:
          self.chessOnField.remove(c)
        else:
          self.chessInHand.remove(c)
        del self.chesses[c.uniqueID]
        c.onField = False
      print(f"  {self}:{chess}升级{colored('成功','green')}，请选择一个 {chess.star + 1} 星棋子")
      print(f"         选项有：{self.upgrade_dict[chess.id]}")
    else:
      print(f"  {self}:{chess}升级{colored('失败','red')}，请检查棋子持有数量")
        
  upgrade_dict = {1: [4, 5], 2: [6, 7], 3: [8, 9], # 1 star 兔子，蚂蚁，小丑鱼

                  4: [10, 11], 5: [12, 13], # 2 star mammal 4：羊驼， 5:狼 
                  6: [14, 15], 7: [16, 17],  # 2 star insect 6: 瓢虫 ，7: 蜜蜂
                  8: [18, 19], 9: [20, 21], # 2 star marine  8: 食人鱼 9: 海胆

                  10: [23,24], 11: [23,24], # 3 star mammal ranged 麋鹿， 猴
                  12: [23,24], 13: [23,24], # 3 star mammal melee 犀牛， 熊

                  14: [25,26], 15: [25,26], # 3 star insect ranged 蝴蝶， 萤火虫
                  16: [25,26], 17: [25,26], # 3 star insect melee 螳螂， 蝎子
                  
                  18: [27,28], 19: [27,28], # 3 star marine ranged 灯笼鱼， 电鳗
                  20: [27,28], 21: [27,28], 22:[], # 3 star marine ranged 螃蟹，一角鲸，海龟

                  23:[],24:[], # 大象， 老虎
                  25:[], 26:[],  # 独角仙， 蜘蛛
                  27:[], 28:[]} # 章鱼， 鲨鱼
  chess_dict = {1: rabbit, 2:ant, 3:littleUglyFish, 
                
                4:llama, 5:wolf, 
                6:ladybug, 7: bee, 
                8: swallower, 9:sea_hedgehog,  
                
                10: heal_deer, 11: monkey, 
                12: hippo, 13: bear,
                14: butterfly, 15: fireworm,
                16: mantis, 17: scorpion, # 3 star insect melee 螳螂， 蝎子
                  
                18: anglerfish, 19: electric_eel, # 3 star marine ranged 灯笼鱼， 电鳗
                20: crab, 21: monoceros, 22: turtle, # 3 star marine ranged 螃蟹，一角鲸，海龟

                23:elephant,24:tiger, # 大象， 老虎
                25:unicorn_b, 26:spider,  # 独角仙， 蜘蛛
                27:octopus, 28:shark} # 章鱼， 鲨鱼
    
  # def get_chess(self,chess:chessInterface):
  #   if chess.id in [1,2,3]:
  #     self.goldAvail -= 1
  #   elif chess.id in [4,5,6,7,8,9]:
  #     self.goldAvail -= 2

  

In [2]:
# 初始化玩家
player1 = Player(0)
player2 = Player(1)

回合1


每人1棋子

In [3]:
player1.printStatus()
player2.printStatus()
player1.get_random_chess_to_draw()
player2.get_random_chess_to_draw()




当前回合 1, 剩余血量：100
玩家0 现在拥有 1 人口，距离下次升级还需要1回合，或者1金钱；
现在拥有的棋子为{}
备战区棋子：[]，上场棋子[]；
当前回合还有可用金钱1；
玩家buff有[]；



当前回合 1, 剩余血量：100
玩家1 现在拥有 1 人口，距离下次升级还需要1回合，或者1金钱；
现在拥有的棋子为{}
备战区棋子：[]，上场棋子[]；
当前回合还有可用金钱1；
玩家buff有[]；


玩家0 本次选择有 [<class 'chess_info.ant'>, <class 'chess_info.littleUglyFish'>, <class 'chess_info.littleUglyFish'>]
玩家1 本次选择有 [<class 'chess_info.littleUglyFish'>, <class 'chess_info.rabbit'>, <class 'chess_info.ant'>]


[chess_info.littleUglyFish, chess_info.rabbit, chess_info.ant]

In [4]:
# 选择棋子
player1.get_chess(littleUglyFish())
player2.get_chess(rabbit())

player1.printChesses()
player2.printChesses()


玩家0 购买棋子小丑鱼1 成功,共花费:  1  金币，还剩  0  金币。
玩家1 购买棋子兔子2 成功,共花费:  1  金币，还剩  0  金币。
{1: 小丑鱼1}
{2: 兔子2}


In [5]:
# 准备阶段
player1Chess = 1
player2Chess = 2

# 上场棋子
player1.hand_to_field(player1Chess, position=[2,1])
player2.hand_to_field(player2Chess, position=[3,2])

# 下场棋子

# player1.field_to_hand(player1Chess)
# player2.field_to_hand(player2Chess)

# player1.printStatus()
# player2.printStatus()


1 <class 'int'> {1: 小丑鱼1}
玩家0:小丑鱼1上场到位置[2, 1]
       备战区棋子：[]，
       上场棋子[小丑鱼1]；
2 <class 'int'> {2: 兔子2}
玩家1:兔子2上场到位置[3, 2]
       备战区棋子：[]，
       上场棋子[兔子2]；


In [6]:
# 战斗阶段
round1Battle = battle()
round1Battle.addRedTeam(redTeam= player1.chessOnField)
round1Battle.addBlueTeam(blueTeam= player2.chessOnField)
round1Battle.battle_with_skills()



[None, None, None, None, None]

[None, None, None, None, None]

[None, 小丑鱼1, None, None, None]

[None, None, 兔子2, None, None]

[None, None, None, None, None]

[None, None, None, None, None]


0.05   兔子2开始从[3, 2]移动到[2, 1] 
0.05 <兔子2>向<up> 移动了一格

[None, None, None, None, None]

[None, None, None, None, None]

[None, 小丑鱼1, 兔子2, None, None]

[None, None, None, None, None]

[None, None, None, None, None]

[None, None, None, None, None]


0.4   兔子2的状态【移动】结束
0.9 <小丑鱼1> 攻击了<兔子2>, 造成了23.204419889502763点伤害，<兔子2>生命值还剩:294.7955801104972 / 318
1.25 <兔子2> 攻击了<小丑鱼1>, 造成了19.35483870967742点伤害，<小丑鱼1>生命值还剩:196.6451612903226 / 216
1.8 <小丑鱼1> 攻击了<兔子2>, 造成了23.204419889502763点伤害，<兔子2>生命值还剩:271.5911602209944 / 318
2.15 <兔子2> 攻击了<小丑鱼1>, 造成了19.35483870967742点伤害，<小丑鱼1>生命值还剩:177.29032258064518 / 216
2.7 <小丑鱼1> 攻击了<兔子2>, 造成了23.204419889502763点伤害，<兔子2>生命值还剩:248.38674033149167 / 318
3.05 <兔子2> 攻击了<小丑鱼1>, 造成了19.35483870967742点伤害，<小丑鱼1>生命值还剩:157.93548387096777 / 216
3.6 <小丑鱼1> 攻击了<兔子2>, 造成了23.204419889502763点伤害，<兔子2>

回合2


In [7]:
player1.next_round()
player2.next_round()

player1.get_random_chess_to_draw()
player2.get_random_chess_to_draw()

玩家0 现在拥有 2 人口，升级至下一人口还需要2回合，或者 4 金钱；


当前回合 2, 剩余血量：90
玩家0 现在拥有 2 人口，距离下次升级还需要2回合，或者4金钱；
现在拥有的棋子为{1: 小丑鱼1}
备战区棋子：[]，上场棋子[小丑鱼1]；
当前回合还有可用金钱2；
玩家buff有[]；


玩家1 现在拥有 2 人口，升级至下一人口还需要2回合，或者 4 金钱；


当前回合 2, 剩余血量：90
玩家1 现在拥有 2 人口，距离下次升级还需要2回合，或者4金钱；
现在拥有的棋子为{2: 兔子2}
备战区棋子：[]，上场棋子[兔子2]；
当前回合还有可用金钱2；
玩家buff有[]；


玩家0 本次选择有 [<class 'chess_info.ant'>, <class 'chess_info.littleUglyFish'>, <class 'chess_info.ant'>]
玩家1 本次选择有 [<class 'chess_info.rabbit'>, <class 'chess_info.littleUglyFish'>, <class 'chess_info.littleUglyFish'>]


[chess_info.rabbit, chess_info.littleUglyFish, chess_info.littleUglyFish]

In [8]:
player1.get_chess(ant())
player1.get_chess(ant())
player2.get_chess(ant())
player2.get_chess(ant())

玩家0 购买棋子蚂蚁3 成功,共花费:  1  金币，还剩  1  金币。
玩家0 购买棋子蚂蚁4 成功,共花费:  1  金币，还剩  0  金币。
玩家1 购买棋子蚂蚁5 成功,共花费:  1  金币，还剩  1  金币。
玩家1 购买棋子蚂蚁6 成功,共花费:  1  金币，还剩  0  金币。


In [9]:
# print(player1.chesses[3])
player1.upgrade_chess(chessID = 3)

3 <class 'int'> {1: 小丑鱼1, 3: 蚂蚁3, 4: 蚂蚁4}
蚂蚁3 <class 'chess_info.ant'> {1: 小丑鱼1, 3: 蚂蚁3, 4: 蚂蚁4}


KeyError: '\x1b[31m玩家0\x1b[0m没有这张卡'

回合3


In [ ]:
# player1.printStatus()
# player2.printStatus()
# player1.get_random_chess_to_draw()
# player2.get_random_chess_to_draw()
# player1.get_chess()
# player2.get_chess()

# player1.redraw()
# player2.upgrade_pop()
# player1.upgrade_chess()

回合4


回合5


回合6


回合7


In [ ]:
    # [ [0,0] [0,1] [0,2] [0,3] [0,4] ]
    # [ [1,0] [1,1] [1,2] [1,3] [1,4] ]
    # [ [2,0] [2,1] [2,2] [2,3] [2,4] ]
    
    # [ [3,0] [3,1] [3,2] [3,3] [3,4] ]
    # [ [4,0] [4,1] [4,2] [4,3] [4,4] ]
    # [ [5,0] [5,1] [5,2] [5,3] [5,4] ]
# blueTeam = [bear(position = [3,2]), bee(position=[5,4]),sea_hedgehog(position=[3,4])]